In [ ]:
import numpy as np
import time
import pandas as pd
import cvxpy as cp
from scipy.optimize import linprog
from scipy.sparse import random as sparse_random
from numpy.linalg import pinv, inv, matrix_rank

def generate_feasible_problems(m=1000, n=1400, num_problems=10, density=1.0):
    problems = []
    for _ in range(num_problems):
        A_sparse = sparse_random(m, n, density=density, data_rvs=np.random.rand)
        A = A_sparse.toarray()
        x_true = np.random.rand(n)
        b = A @ x_true
        c = np.ones(n)
        problems.append((A, b, c))
    return problems

def generate_random_matrix(m, k):
    # Generate a k x m Gaussian random matrix with entries ~ N(0, 1/sqrt(k))
    return np.random.normal(0, 1/np.sqrt(k), size=(k, m))

def solve_projected_LP(A, b, c, k):
    """
    Solve the projected LP:  min c^T x  subject to T A x = T b, x >= 0.
    Returns:
      bar_x : primal solution of the projected LP
      lam   : dual variables for T A x = T b
      T     : random projection matrix
      time_sample_T : time to sample T
      time_multiply_TA : time to multiply T by A
      time_solve : time to solve the projected LP
    """
    m, n = A.shape

    # Time sampling T
    t_sample = time.perf_counter()
    T = generate_random_matrix(m, k)
    time_sample_T = time.perf_counter() - t_sample

    # Time multiplying T by A
    t_multiply = time.perf_counter()
    TA = T @ A  # Compute T A
    Tb = T @ b  # Compute T b
    time_multiply_TA = time.perf_counter() - t_multiply

    # Time solving the projected LP
    t_solve = time.perf_counter()
    x = cp.Variable(n, nonneg=True)
    constraints = [TA @ x == Tb]
    objective = cp.Minimize(c @ x)
    prob = cp.Problem(objective, constraints)
    prob.solve()
    time_solve = time.perf_counter() - t_solve

    if prob.status != 'optimal':
        raise ValueError("Projected LP did not solve optimally")

    bar_x = x.value
    lam = constraints[0].dual_value  # dual variable for T A x = T b
    return bar_x, lam, T, time_sample_T, time_multiply_TA, time_solve

def retrieve_solution_algorithm1(A, b, c, bar_x, lam, T):
    """
    Implements Algorithm 1 from the screenshot:
      1) Compute y_prox = T^T lam
      2) For each j, compute z_j = (c_j - A_j^T y_prox) / ||A_j||
      3) Select the m columns with smallest z_j; call that set B
      4) Solve x_B = A_B^{-1} b (using inverse, not pseudoinverse) and embed in full dimension
    """
    m, n = A.shape
    y_prox = T.T @ lam

    # Distance-like metric z_j
    z = np.zeros(n)
    for j in range(n):
        col_norm = np.linalg.norm(A[:, j], 2)
        if col_norm > 0:
            z[j] = (c[j] - A[:, j].T @ y_prox) / col_norm
        else:
            z[j] = np.inf  # Skip zero columns, as in original code

    # Pick the m smallest z_j
    idx_B = np.argsort(z)[:m]
    A_B = A[:, idx_B]

    # Check if A_B is invertible (i.e., full rank and m x m)
    if matrix_rank(A_B) != m:
        raise ValueError("Selected submatrix A_B is not invertible (columns are linearly dependent)")

    # Solve with inverse (not pseudoinverse) to match the screenshot
    x_B = inv(A_B) @ b

    # Embed in full dimension
    x_full = np.zeros(n)
    x_full[idx_B] = x_B
    return x_full

def retrieve_solution_algorithm2(A, b, bar_x):
    """
    Algorithm 2 (fixed):
    1) Let H be the m indices of bar_x with largest values (the projected basis).
    2) Solve A_H^T A_H x_H = A_H^T b by pseudoinverse.
    3) Embed into full x.
    """
    m, n = A.shape

    # 1) pick the m indices of bar_x with largest entries
    idx_H = np.argsort(bar_x)[-m:]
    A_H = A[:, idx_H]

    # optional check: ensure A_H is full‑rank
    if matrix_rank(A_H) != m:
        raise ValueError("Projected basis A_H is rank-deficient")

    # 2) normal-equation solve via pseudoinverse
    x_H = pinv(A_H.T @ A_H) @ (A_H.T @ b)

    # 3) embed in full dimension
    x_full = np.zeros(n)
    x_full[idx_H] = x_H
    return x_full


def random_projection_LP(A, b, c, k):
    """
    1) Solve projected LP => bar_x, lam, T
    2) Retrieve solutions with both algorithms
    Returns:
      x_alg1 : solution from Algorithm 1
      x_alg2 : solution from Algorithm 2
      prjCPU1 : total time for projection steps + Algorithm 1 retrieval
      prjCPU2 : total time for projection steps + Algorithm 2 retrieval
    """
    # Solve the projected LP and get timing components
    bar_x, lam, T, time_sample_T, time_multiply_TA, time_solve = solve_projected_LP(A, b, c, k)

    # Time retrieval for Algorithm 1
    t_retrieval1 = time.perf_counter()
    x_alg1 = retrieve_solution_algorithm1(A, b, c, bar_x, lam, T)
    time_retrieval1 = time.perf_counter() - t_retrieval1

    # Time retrieval for Algorithm 2
    t_retrieval2 = time.perf_counter()
    x_alg2 = retrieve_solution_algorithm2(A, b, bar_x)
    time_retrieval2 = time.perf_counter() - t_retrieval2

    # Compute total times for prjCPU1 and prjCPU2
    prjCPU1 = time_sample_T + time_multiply_TA + time_solve + time_retrieval1
    prjCPU2 = time_sample_T + time_multiply_TA + time_solve + time_retrieval2

    return x_alg1, x_alg2, prjCPU1, prjCPU2

# Example experiment
m, n, num_problems, k = 1000, 1400, 10, 327
densities = [0.1, 0.3, 0.5, 0.7]
experiment_results = []

for density in densities:
    print(f"=== Running experiments with density {density} ===")
    problems = generate_feasible_problems(m, n, num_problems, density=density)

    total_orig_time = 0.0
    total_prjCPU1 = 0.0
    total_prjCPU2 = 0.0

    total_feas1 = 0.0
    total_objerror1 = 0.0
    total_neg1 = 0.0

    total_feas2 = 0.0
    total_objerror2 = 0.0
    total_neg2 = 0.0

    for i, (A, b, c) in enumerate(problems, start=1):
        # Solve original LP (orgCPU)
        t0 = time.perf_counter()
        res_orig = linprog(c, A_eq=A, b_eq=b, bounds=(0, None), method='highs')
        orig_time = time.perf_counter() - t0

        # Solve projected LP + retrieve (prjCPU1 and prjCPU2)
        x_alg1, x_alg2, prjCPU1, prjCPU2 = random_projection_LP(A, b, c, k)

        # Metrics
        feas1 = np.linalg.norm(A @ x_alg1 - b, 1) / np.linalg.norm(b, 1)
        obj1 = c @ x_alg1
        objerror1 = (abs(res_orig.fun - obj1) / abs(res_orig.fun)
                     if res_orig.fun != 0 else np.nan)
        neg1 = (np.sum(np.abs(x_alg1[x_alg1 < 0])) / np.linalg.norm(x_alg1, 1)
                if np.linalg.norm(x_alg1, 1) != 0 else np.nan)

        feas2 = np.linalg.norm(A @ x_alg2 - b, 1) / np.linalg.norm(b, 1)
        obj2 = c @ x_alg2
        objerror2 = (abs(res_orig.fun - obj2) / abs(res_orig.fun)
                     if res_orig.fun != 0 else np.nan)
        neg2 = (np.sum(np.abs(x_alg2[x_alg2 < 0])) / np.linalg.norm(x_alg2, 1)
                if np.linalg.norm(x_alg2, 1) != 0 else np.nan)

        print(f"Problem {i}:")
        print(f"  Original LP: status={res_orig.status}, orgCPU={orig_time:.4f}s, obj={res_orig.fun:.4f}")
        print(f"  Alg.1: prjCPU1={prjCPU1:.4f}s, obj={obj1:.4f}, feas={feas1:.4e}, objerr={objerror1:.4e}, neg={neg1:.4e}")
        print(f"  Alg.2: prjCPU2={prjCPU2:.4f}s, obj={obj2:.4f}, feas={feas2:.4e}, objerr={objerror2:.4e}, neg={neg2:.4e}\n")

        total_orig_time += orig_time
        total_prjCPU1 += prjCPU1
        total_prjCPU2 += prjCPU2

        total_feas1 += feas1
        total_objerror1 += objerror1
        total_neg1 += neg1

        total_feas2 += feas2
        total_objerror2 += objerror2
        total_neg2 += neg2

    # Averages
    avg_orig_time = total_orig_time / num_problems
    avg_prjCPU1 = total_prjCPU1 / num_problems
    avg_prjCPU2 = total_prjCPU2 / num_problems

    avg_feas1 = total_feas1 / num_problems
    avg_objerror1 = total_objerror1 / num_problems
    avg_neg1 = total_neg1 / num_problems

    avg_feas2 = total_feas2 / num_problems
    avg_objerror2 = total_objerror2 / num_problems
    avg_neg2 = total_neg2 / num_problems

    print("Average results:")
    print(f"  Original LP avg orgCPU = {avg_orig_time:.4f}s")
    print(f"  Alg.1 avg prjCPU1 = {avg_prjCPU1:.4f}s, feas={avg_feas1:.4e}, objerr={avg_objerror1:.4e}, neg={avg_neg1:.4e}")
    print(f"  Alg.2 avg prjCPU2 = {avg_prjCPU2:.4f}s, feas={avg_feas2:.4e}, objerr={avg_objerror2:.4e}, neg={avg_neg2:.4e}\n")

    experiment_results.append((
        m, n, density, k,
        avg_orig_time, avg_prjCPU1, avg_prjCPU2,
        avg_feas1, avg_objerror1, avg_neg1,
        avg_feas2, avg_objerror2, avg_neg2
    ))

df = pd.DataFrame(
    experiment_results,
    columns=[
        "m", "n", "density", "k", "orgCPU", "prjCPU1", "prjCPU2",
        "feasibility error1", "objective error1", "negativity1",
        "feasibility error2", "objective error2", "negativity2"
    ]
)
print(df)

=== Running experiments with density 0.1 ===
Problem 1:
  Original LP: status=0, orgCPU=31.9315s, obj=680.0965
  Alg.1: prjCPU1=4.7902s, obj=803.8959, feas=5.0111e-13, objerr=1.8203e-01, neg=4.7780e-01
  Alg.2: prjCPU2=6.6725s, obj=634.1224, feas=4.8692e-10, objerr=6.7599e-02, neg=4.8354e-01

Problem 2:
  Original LP: status=0, orgCPU=32.3542s, obj=676.4854
  Alg.1: prjCPU1=6.8267s, obj=642.0097, feas=1.3168e-12, objerr=5.0963e-02, neg=4.9541e-01
  Alg.2: prjCPU2=6.8093s, obj=608.5226, feas=1.4120e-10, objerr=1.0046e-01, neg=4.8332e-01

Problem 3:
  Original LP: status=0, orgCPU=28.9439s, obj=681.8363
  Alg.1: prjCPU1=6.3360s, obj=720.7736, feas=7.9016e-13, objerr=5.7107e-02, neg=4.7972e-01
  Alg.2: prjCPU2=8.9616s, obj=584.2473, feas=1.0192e-08, objerr=1.4313e-01, neg=4.9693e-01

Problem 4:
  Original LP: status=0, orgCPU=32.3354s, obj=688.5827
  Alg.1: prjCPU1=6.1571s, obj=577.3229, feas=2.9368e-12, objerr=1.6158e-01, neg=4.9871e-01
  Alg.2: prjCPU2=6.8615s, obj=695.7114, feas=3.6738

In [ ]:
# Ensure a global list exists to store results across runs
try:
    experiment_results
except NameError:
    experiment_results = []

# After computing the averages for the current (m, n, density, k),
# for example:
# avg_orig_time, avg_prjCPU1, avg_prjCPU2,
# avg_feas1, avg_objerror1, avg_neg1,
# avg_feas2, avg_objerror2, avg_neg2

experiment_results.append((
    m, n, density, k,
    avg_orig_time,  # Time for original LP (orgCPU)
    avg_prjCPU1,    # Time for projection steps + Algorithm 1 (prjCPU1)
    avg_prjCPU2,    # Time for projection steps + Algorithm 2 (prjCPU2)
    avg_feas1,      # Feasibility error for Algorithm 1
    avg_objerror1,  # Objective error for Algorithm 1
    avg_neg1,       # Negativity for Algorithm 1
    avg_feas2,      # Feasibility error for Algorithm 2
    avg_objerror2,  # Objective error for Algorithm 2
    avg_neg2        # Negativity for Algorithm 2
))
print("Stored result for (m, n, dens, k):", (m, n, density, k))


Stored result for (m, n, dens, k): (1000, 1400, 0.7, 327)


In [ ]:
import pandas as pd

# Create a DataFrame from the accumulated results
df = pd.DataFrame(experiment_results, columns=[
    "m", "n", "dens", "k",
    "orgCPU",
    "recoveredCPU1",
    "recoveredCPU2",
    "feas1",
    "obj1",
    "neg1",
    "feas2",
    "obj2",
    "neg2"
])

print(df)



      m     n  dens    k     orgCPU  recoveredCPU1  recoveredCPU2  \
0  1000  1400   0.1  327  31.300586       6.035726       6.984326   
1  1000  1400   0.3  327  37.237293       5.759524       6.575257   
2  1000  1400   0.5  327  45.279461       6.729259       7.512893   
3  1000  1400   0.7  327  44.556952       6.397418       6.957540   
4  1000  1400   0.7  327  44.556952       6.397418       6.957540   

          feas1      obj1      neg1         feas2      obj2      neg2  
0  1.195103e-12  0.097721  0.480357  3.815628e-09  0.199368  0.485709  
1  1.186358e-11  0.435347  0.490650  9.993865e-09  0.141716  0.483667  
2  2.273709e-12  0.230906  0.482178  1.941725e-09  0.027769  0.479407  
3  2.342817e-12  0.137283  0.486186  8.178549e-08  0.023692  0.479514  
4  2.342817e-12  0.137283  0.486186  8.178549e-08  0.023692  0.479514  
